In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, root_mean_squared_error,r2_score
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from pytorch_tabnet.tab_model import TabNetRegressor

from hyperopt import fmin, tpe, hp, Trials

import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from pre_processing import preprocess_data

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [2]:
def load_data():
    dataset = pd.read_excel(r"globalterrorismdb_2021Jan-June_1222dist.xlsx") # 2021-2021 June
    return dataset


## Data Loading and Preprocessing

In [3]:
dataset = load_data()
X, y, dataset = preprocess_data(dataset)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
dataset.head()


,success,gname_freq,city_freq,country_freq,attacktype1_score,targtype1_score,weaptype1_score,gname_score,country_score,city_score,...,nkill_likelihood_score,region_3,region_5,region_6,region_8,region_9,region_10,region_11,region_12,nkill
0,1.0,0.228571,0.066667,0.167228,0.875,1.000000,0.857143,0.977778,0.948718,0.123077,...,0.589934,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0
1,1.0,0.121429,0.033333,0.096521,0.500,0.842105,0.857143,0.911111,0.871795,0.030769,...,0.347526,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,1.0,0.000000,0.033333,0.159371,0.875,0.894737,0.857143,0.000000,0.641026,0.015385,...,0.477050,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.121429,0.033333,0.096521,0.875,0.894737,0.857143,0.911111,0.871795,0.092308,...,0.503175,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6.0
4,1.0,0.000000,0.033333,0.093154,0.875,1.000000,0.857143,0.022222,0.743590,0.030769,...,0.543654,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


## 3) MLP
## Hyperparameter Tuning and Model Training

In [4]:
class TerrorismDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).view(-1, 1)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim1 = 128, hidden_dim2 = 64, dropout_rate = 0.2, activation_function='ELU'):
        super(MLP, self).__init__()
        
        if activation_function == 'ReLU':
            self.activation = nn.ReLU()
        elif activation_function == 'ELU':
            self.activation = nn.ELU()
        elif activation_function == 'LeakyReLU':
            self.activation = nn.LeakyReLU()
            
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(hidden_dim2, 1)

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.dropout1(x)
        x = self.activation(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x


In [5]:
# Convert to Dataset and DataLoader
train_dataset = TerrorismDataset(X_train, y_train)
test_dataset = TerrorismDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


def train_model(model, train_loader, val_loader, num_epochs=500, lr=0.005, weight_decay=1e-4, patience=15):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    best_rmse = float('inf')
    patience_counter = 0

    # Training Loop with Early Stopping
    for epoch in range(num_epochs):
        model.train()

        for X_train, y_train in train_loader:
            X_train, y_train = X_train.to(device), y_train.to(device)

            optimizer.zero_grad()
            outputs = model(X_train)
            loss = criterion(outputs, y_train)
            loss.backward()
            optimizer.step()

        # Validation Step
        model.eval()
        val_preds, val_labels = [], []

        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                val_outputs = model(X_val)
                val_preds.extend(val_outputs.cpu().numpy())
                val_labels.extend(y_val.cpu().numpy())

        # Calculate RMSE
        val_rmse = np.sqrt(np.mean((np.array(val_preds) - np.array(val_labels)) ** 2))

        # Early Stopping Check
        if val_rmse < best_rmse:
            best_rmse = val_rmse
            patience_counter = 0  # Reset patience counter
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch}. Best RMSE: {best_rmse:.4f}")
            break

    return best_rmse

def objective(params):
    hidden_dim1 = int(params['hidden_dim1'])
    hidden_dim2 = int(params['hidden_dim2'])
    dropout_rate = params['dropout_rate']
    lr = params['lr']
    weight_decay = params['weight_decay']
    activation_function = params['activation_function']

    # Initialize the model with the current hyperparameters
    model = MLP(input_dim=X_train.shape[1],
                hidden_dim1=hidden_dim1,
                hidden_dim2=hidden_dim2,
                activation_function= activation_function,
                dropout_rate=dropout_rate).to(device)

    # Train the model and get validation RMSE
    val_rmse = train_model(model, train_loader, val_loader, num_epochs=500, patience = 50,  lr=lr, weight_decay=weight_decay)

    return val_rmse  # Minimizing the RMSE loss

# Define the search space
space = {
    'hidden_dim1': hp.quniform('hidden_dim1', 32, 512, 16),
    'hidden_dim2': hp.quniform('hidden_dim2', 16, 256, 16),
    'dropout_rate': hp.uniform('dropout_rate', 0.1, 0.5),
    'lr': hp.loguniform('lr', -5, -1),
    'weight_decay': hp.loguniform('weight_decay', -5, -1),
    'activation_function': hp.choice('activation_function', ['ReLU', 'ELU', 'LeakyReLU'])

}

# Initialize Trials object to store results
trials = Trials()

# Run the optimization
best_params = fmin(
    fn=objective,             # Objective function
    space=space,              # Search space
    algo=tpe.suggest,         # Optimization algorithm
    max_evals=200,             # Number of evaluations
    trials=trials             # Store trials
)

print("Best hyperparameters found:", best_params)


Early stopping at epoch 55. Best RMSE: 6.3904          
Early stopping at epoch 101. Best RMSE: 7.0760                                   
Early stopping at epoch 164. Best RMSE: 3.3805                                   
Early stopping at epoch 117. Best RMSE: 7.4871                                   
Early stopping at epoch 122. Best RMSE: 3.4562                                  
Early stopping at epoch 51. Best RMSE: 6.6373                                   
Early stopping at epoch 53. Best RMSE: 4.9789                                   
Early stopping at epoch 118. Best RMSE: 6.4934                                  
Early stopping at epoch 61. Best RMSE: 4.4675                                   
Early stopping at epoch 82. Best RMSE: 4.8670                                   
Early stopping at epoch 67. Best RMSE: 4.4131                                    
Early stopping at epoch 90. Best RMSE: 4.9318                                    
Early stopping at epoch 165. Best RMSE: 3.3290  